
# Projeto 2 - SCC0252 (Visualização Computacional)


*   Otávio Ferracioli Coletti - 11767796
*   Fernando César Lopes Barbosa Filho - 10260559

\\

---

In [26]:
# Bibliotecas importadas
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import Button, Layout
from matplotlib.spines import rcParams
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib import legend
from pandas import Series
import ipywidgets
import random
import plotly
import io

In [27]:
# Conjunto de dados do Spotify
url = 'https://raw.githubusercontent.com/otaviofcoletti/VIZCOMP/main/SpotifyFeatures(1).csv'
df = pd.read_csv(url)

## Heatmap

Geramos um *heatmap* que exibe os valores de correlação entre os atributos de todas as músicas do Spotify.


In [28]:
popular_music_rank = df.sort_values('popularity', ascending=False) 

In [29]:
def heatmap_interactive(**ch_list):

  categories = ['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness']
  features = []

  for i in categories:
    if ch_list[i]:
      features.append(i)

  selection = df[features]
  plt.figure(figsize=(15,10))
  sns.heatmap(selection.corr(),annot=True,cmap="coolwarm")
  plt.title('Heatmap of Correlations')
  plt.show()

In [30]:
cb_list = {description: widgets.Checkbox(
    value=random.choice([True, False]),
    description=description,
    disabled=False,
    indent=False
) for description in ['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness']}

Selecione nos **checkboxes** as *features* nas quais se deseja comparar a correlação no **heatmap**:

In [ ]:
#widgets.interactive(heatmap_interactive, **cb_list)

## Gráfico de Radar

Nesse tipo de visualização o usuário poderá selecionar quantas músicas se deseja analisar entre as mais populares, quais as *features* a serem os eixos e a respectiva ordem entre elas.

In [32]:
cinza = "#808080"
transparente = "rgba(0,0,0,0)"

def radar_chart_2(number_of_music=1,**dd_list):

  top5 = popular_music_rank[:number_of_music]

  lista_features = ['Feature 1','Feature 2','Feature 3','Feature 4','Feature 5','Feature 6','Feature 7','Feature 8']
  categories = ['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness']
  top5 = top5[['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness','track_name','artist_name']]
  fig = go.Figure()
  features = []

  for i in lista_features:

    if dd_list[i] != "Nenhum":
      features.append(dd_list[i])
  

  for i in range(0,number_of_music):

    fig.add_trace(go.Scatterpolar(
          r=list(top5.iloc[i]),
          theta=features,
          fill='toself',
          name= top5.iloc[i]['track_name'],
          legendgrouptitle_text="Músicas mais populares"
    ))


  fig.update_layout(
        font_color=cinza,
        font_size=15,
        autosize=True,
        height=400,
        margin=dict(l=40, r=40, t=25, b=25),
        paper_bgcolor=transparente,
        polar=dict(
            bgcolor=transparente,
            radialaxis=dict(
                range=[0, 1],
                linecolor=cinza,
                gridcolor=cinza,
                tickcolor=cinza,
            ),
            angularaxis=dict(
                rotation=120,
                direction="clockwise",
                linecolor=cinza,
                tickcolor=cinza,
                gridcolor=cinza,
            ),
        ),
    )
  
  fig.show()

In [33]:
lista_features = ['Feature 1','Feature 2','Feature 3','Feature 4','Feature 5','Feature 6','Feature 7','Feature 8']

In [34]:
charac=['valence','acousticness','danceability','energy','liveness','speechiness','instrumentalness','loudness','Nenhum']

In [35]:
index_list = [0,1,2,3,4,5,6,7,8]
random.seed(117)

dd_radar = {descri: widgets.Dropdown(
    options=charac,
    value=charac[index],
    description=descri,
    disabled=False,
) for index,descri in zip(index_list,lista_features) }

Através do gráfico de radar interativo podemos escolher quais as *features* musicais de interesse na análise, bem como escolher quantas e quais músicas comparar clicando na legenda associada. O usuário poderá escolher até 8 features, mas recomendamos entre 5 e 6 para facilitar a visualização.

In [ ]:
ipywidgets.interactive(radar_chart_2, number_of_music=ipywidgets.IntSlider(min=1,max=15,step=1,value=5),**dd_radar)

Podemos ver que, das músicas mais populares, todas possuem um *loudness* alto e uma *danceability* significativamente alta também. Notamos que, por exemplo, a acústica dessas músicas dificilmente ultrapassa 5. \\

Portanto, para ter sucesso é preciso fazer barulho, fazer o pessoal dançar e usar bastante da eletrônica na sua música! \\


## Treemap

Esse recurso nos permite visualizar a hierarquia de popularidade entre as diferentes músicas e artistas.

In [37]:
def treemap_plot(artist_or_genre, size_selection=10):

  if(artist_or_genre == 'Hierarquia por Artista'):
    fig = px.treemap(popular_music_rank[0:size_selection], path=['artist_name','track_name'],color='popularity', values ='popularity',title="Hierarquia das popularidades entre artistas",hover_name='artist_name',width=900, height=700)
  else:
    fig = px.treemap(popular_music_rank[0:size_selection], path=['genre', 'artist_name'], values ='popularity',title="Participação de cada artista na popularidade de cada gênero musical",hover_name='track_name',width=900, height=700)
    #fig = px.treemap(popular_music_rank[0:size_selection], path=['genre'], values ='popularity',title="Hierarquia das popularidades entre artistas",hover_name='genre',width=900, height=700)

  fig.update_traces(root_color="lightgrey")
  fig.update_traces(legendgrouptitle_text="Músicas mais populares")
  fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
  fig.update_layout(
    font_family="Helvetica",
    font_color="black",
  )
  fig.update_xaxes(title_font_family="Helvetica")
  fig.show()

In [ ]:
ipywidgets.interact(treemap_plot, artist_or_genre=widgets.Dropdown(options=['Hierarquia por Artista','Hierarquia por Gênero Musical'],description='Selecione:'), size_selection=ipywidgets.IntSlider(min=5, max=100,step=5,value=25,description="N° músicas"))